In [6]:
import set_env  # not included in the repo, defines $PROJECT_DIR, $PNC_PROJECT_DIR, $IMAGEN_PROJECT_DIR & $INPUT_DIR

In [7]:
import os
import glob
import sys
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt

OUTPUT_DIR = {
    "pnc": os.path.join(os.environ["PNC_PROJECT_DIR"], "output"),
    "imagen": os.path.join(os.environ["IMAGEN_PROJECT_DIR"], "output"),
    "micamics": os.path.join(os.environ["PROJECT_DIR"], "output", "micamics")
}
INPUT_DIR = os.environ["INPUT_DIR"]
CODE_DIR = os.path.join(os.getcwd(), "..", "scripts")
sys.path.append(CODE_DIR)
from utils import stats

from modeling.check_fit import load_cmaes

import warnings
warnings.filterwarnings("ignore")

plt.rcParams.update({"font.size": 12, "font.family": "Asana Math", "figure.dpi": 192})
thcolor = "#44546A"

Load data of the ground truth (synthetic) simulation:

In [8]:
orig_data = np.load('./ground_truth/synthetic/it1.npz', allow_pickle=True)
orig_vars = {}
# parameters
for p in ['wEE', 'wEI', 'wIE']:
    # in cuBNM parameter names are wEE etc.
    orig_vars[p] = orig_data[p].squeeze()
# I_E
orig_vars['I_E'] = orig_data['sim_states'].flatten()[0]['I_E'].squeeze()
orig_vars = pd.DataFrame(orig_vars)

Load data of the optima resulted in the recovery CMAES runs:

In [9]:
recov_opt = {}
recov_data = {}
best_opt_seeds = {}
for sim_seed in [410, 40]:
    recov_opt[sim_seed] = {}
    recov_data[sim_seed] = {}   
    for opt_seed in [1, 2]:
        # load optimum params and gof
        recov_opt[sim_seed][opt_seed] = pd.read_csv(f'./ground_truth/recovery/sim-{sim_seed}_opt-{opt_seed}/cmaes_run-0/opt.csv', index_col=0).iloc[:, 0]
        recov_opt[sim_seed][opt_seed]['opt_seed'] = opt_seed
        # load optimum simulation data
        sim_data = np.load(
            glob.glob(f'./ground_truth/recovery/sim-{sim_seed}_opt-{opt_seed}/cmaes_run-0/opt_sim/it*.npz')[0], 
            allow_pickle=True
        )
        recov_data[sim_seed][opt_seed] = {}
        for param in ['wEE', 'wEI', 'wIE']:
            recov_data[sim_seed][opt_seed][param] = sim_data[param][0]
        recov_data[sim_seed][opt_seed]['I_E'] = sim_data['sim_states'].flatten()[0]['I_E'].flatten()
    best_opt_seed = int(pd.DataFrame(recov_opt[sim_seed]).T.set_index('opt_seed')['+gof'].idxmax())
    best_opt_seeds[sim_seed] = best_opt_seed

Load data of the optima resulted in the recovery CMAES runs:

In [10]:
for sim_seed in [410, 40]:
    print(f"{'-'*10}\nsim_seed: {sim_seed}\n{'-'*10}")
    best_opt_seed = best_opt_seeds[sim_seed]
    print("GOF measures")
    for measure in ['+gof', '+fc_corr', '-fcd_ks', '-fc_diff']:
        print(f"\t{measure}: {recov_opt[sim_seed][best_opt_seed][measure]:.3f}")
    for var in ['wEE', 'wEI', 'wIE', 'I_E']:
        r = scipy.stats.pearsonr(recov_data[sim_seed][best_opt_seed][var], orig_vars[var])
        madicc = stats.madicc(recov_data[sim_seed][best_opt_seed][var], orig_vars[var])
        print(f"{var}\n\tCorrelation = {r.statistic:.3f} (p = {r.pvalue:.3f})\n\tmadICC = {madicc:.3f}")
    print(f"G\n\tTruth: {orig_data['G'][0]}; Recovered: {recov_opt[sim_seed][best_opt_seed]['G']:.2f}")

----------
sim_seed: 410
----------
GOF measures
	+gof: 0.966
	+fc_corr: 0.992
	-fcd_ks: -0.019
	-fc_diff: -0.007
wEE
	Correlation = 0.299 (p = 0.003)
	madICC = -0.215
wEI
	Correlation = 0.937 (p = 0.000)
	madICC = 0.922
wIE
	Correlation = 0.957 (p = 0.000)
	madICC = 0.915
I_E
	Correlation = 0.977 (p = 0.000)
	madICC = 0.943
G
	Truth: 2.5; Recovered: 2.69
----------
sim_seed: 40
----------
GOF measures
	+gof: 0.810
	+fc_corr: 0.842
	-fcd_ks: -0.031
	-fc_diff: -0.001
wEE
	Correlation = 0.918 (p = 0.000)
	madICC = 0.863
wEI
	Correlation = 0.925 (p = 0.000)
	madICC = 0.795
wIE
	Correlation = 0.963 (p = 0.000)
	madICC = 0.972
I_E
	Correlation = 0.734 (p = 0.000)
	madICC = 0.472
G
	Truth: 2.5; Recovered: 3.92
